In [1]:
#note this now uses rubin_sim package
# See https://community.lsst.org/t/announcing-rubin-sim/5881


from rubin_sim.maf.metrics.baseMetric import BaseMetric
from scipy import interpolate

import rubin_sim.maf.utils as mafUtils

%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import healpy as hp

import rubin_sim.maf.db as db
import rubin_sim.maf.utils as utils
import rubin_sim.maf.metrics as metrics
import rubin_sim.maf.slicers as slicers
import rubin_sim.maf.metricBundles as metricBundles
import rubin_sim.maf.plots as plots
import rubin_sim.maf.maps as maps
# need a different set of utils for the stellarmags
import rubin_sim.utils as sims_utils


import glob


In [2]:
# Copy and modify the ParallaxMetric

class SRDParallaxZMetric(BaseMetric):
    """Calculate the distance to which one could reach a parallax SNR for a given object

    Uses columns ra_pi_amp and dec_pi_amp, calculated by the ParallaxFactorStacker.

    Parameters
    ----------
    metricName : str, opt
        Default 'parallax'.
    m5Col : str, opt
        The default column name for m5 information in the input data. Default fiveSigmaDepth.
    filterCol : str, opt
        The column name for the filter information. Default filter.
    seeingCol : str, opt
        The column name for the seeing information. Since the astrometry errors are based on the physical
        size of the PSF, this should be the FWHM of the physical psf. Default seeingFwhmGeom.
    mags : dict
        The absolute magnitude of the obeject in question. Keys of filter name, values in mags.
    distances : np.array
        Distances to try putting the object at (pc).
    atm_err : float, opt
        The expected centroiding error due to the atmosphere, in arcseconds. Default 0.01.
    badval : float, opt
        The value to return when the metric value cannot be calculated. Default 0.
    """
    def __init__(self, metricName='bdParallax', m5Col='fiveSigmaDepth',
                 filterCol='filter', seeingCol='seeingFwhmGeom',
                 badval=0, mags={'y': 20.00},  
                 atm_err=0.010, normalize=False, min_exp_time=15., 
                 exptimeCol='visitExposureTime', **kwargs):
        Cols = [m5Col, filterCol, seeingCol, exptimeCol, 'ra_pi_amp', 'dec_pi_amp']
        
        units = 'pc'
        super(SRDParallaxZMetric, self).__init__(Cols, metricName=metricName, units=units,
                                             badval=badval, **kwargs)
        # set return type
        self.m5Col = m5Col
        self.seeingCol = seeingCol
        self.filterCol = filterCol
        self.exptimeCol = exptimeCol
        self.mags = {}
        for key in mags:
            self.mags[key] = mags[key] 
        self.atm_err = atm_err
        self.filters = list(self.mags.keys())
        self.min_exp_time = min_exp_time
        
    def _final_sigma(self, position_errors, ra_pi_amp, dec_pi_amp):
        """Assume parallax in RA and DEC are fit independently, then combined.
        All inputs assumed to be arcsec """
        sigma_A = position_errors/ra_pi_amp
        sigma_B = position_errors/dec_pi_amp
        sigma_ra = np.sqrt(1./np.sum(1./sigma_A**2, axis=1))
        sigma_dec = np.sqrt(1./np.sum(1./sigma_B**2, axis=1))
        # Combine RA and Dec uncertainties, convert to mas
        sigma = np.sqrt(1./(1./sigma_ra**2+1./sigma_dec**2))*1e3
        return sigma

    def run(self, dataslice, slicePoint=None):
        snr = np.zeros((np.size(self.mags[self.filters[0]]), len(dataslice)), dtype='float')
        # compute SNR for all observations
        for filt in self.filters:
            if self.min_exp_time is not None:
                good = np.where((dataslice[self.filterCol] == filt) &
                                (dataslice[self.exptimeCol] >= self.min_exp_time))[0]
            else:
                good = np.where(dataslice[self.filterCol] == filt)[0]
            if np.size(good) > 0:
                snr[:, good] = mafUtils.m52snr(self.mags[str(filt)][:, np.newaxis],
                                               dataslice[self.m5Col][good])
                
        
        position_errors = np.sqrt(mafUtils.astrom_precision(dataslice[self.seeingCol],
                                                            snr)**2+self.atm_err**2)
        # uncertainty in the parallax in mas
        sigma = self._final_sigma(position_errors, dataslice['ra_pi_amp'], dataslice['dec_pi_amp'])
        result = sigma
        return result

In [3]:
class AreaSumMetric(BaseMetric):
    """Compute the total area assuming a metric is in arcsec.
    """
    def __init__(self, col=None, metricName='VolumeSum', nside=None, **kwargs):
        super(AreaSumMetric, self).__init__(col=col, metricName=metricName, **kwargs)
        self.pix_area = hp.nside2pixarea(nside)
        
    def run(self, dataSlice, slicePoint=None):
        # volume of sphere, times ratio of pixel area divided by area of sphere
        if (dataSlice[self.colname] >= 0.006):
            areas =  self.pix_area
        else:
            areas=0.0
        return np.sum(areas)

In [4]:
files = glob.glob('/sims_maf/fbs_2.0/baseline/*.db')
files.sort()
results = []
for dbFile in files:
    
    runName = dbFile.replace('.db', '')
    conn = db.OpsimDatabase(dbFile)
    
    outDir = runName + '_bdvol'
    outDir=outDir.replace('/sims_maf/fbs_2.0/',"SRDYresults/")
    resultsDb = db.ResultsDb(outDir=outDir)
    
    nside = 64
    bundleList = []
    #sql = 'filter="z" or filter="y"'
    sql = ''
    # Let's plug in the magnitudes for one type
    metric = SRDParallaxZMetric(mags={'y': 20.00}, metricName='Z20')
    slicer = slicers.HealpixSlicer(nside=nside)
    summaryStats = [AreaSumMetric(nside=nside)]
    plotDict = {}
    bundleList.append(metricBundles.MetricBundle(metric,slicer,sql, plotDict=plotDict,
                                                 summaryMetrics=summaryStats,
                                                 runName=runName))
    bd = metricBundles.makeBundlesDictFromList(bundleList)
    bg = metricBundles.MetricBundleGroup(bd, conn, outDir=outDir, resultsDb=resultsDb)
    bg.runAll()
    bg.plotAll(closefigs=False)

    results.append(bg)

Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Querying database observations with no constraint for columns ['seeingFwhmGeom', 'fieldRA', 'observationStartMJD', 'fieldDec', 'visitExposureTime', 'filter', 'rotSkyPos', 'fiveSigmaDepth'].
Found 2086980 visits
Running:  ['_sims_maf_fbs_2_0_baseline_baseline_v2_0_10yrs_Z20_HEAL']


/data0/sw/lsst_stack-2021.10.13/rubin_sim/rubin_sim/maf/utils/astrometryUtils.py:72: RuntimeWarning: divide by zero encountered in true_divide
  result = fwhm/(snr)
<ipython-input-2-e5fe2430f7b7>:55: RuntimeWarning: divide by zero encountered in true_divide
  sigma_ra = np.sqrt(1./np.sum(1./sigma_A**2, axis=1))
<ipython-input-2-e5fe2430f7b7>:56: RuntimeWarning: divide by zero encountered in true_divide
  sigma_dec = np.sqrt(1./np.sum(1./sigma_B**2, axis=1))
<ipython-input-2-e5fe2430f7b7>:58: RuntimeWarning: divide by zero encountered in true_divide
  sigma = np.sqrt(1./(1./sigma_ra**2+1./sigma_dec**2))*1e3


TypeError: 'float' object is not subscriptable

In [ ]:
# Let's see how the total area changes as we change the footprint
for name, result in zip(files, results):
    key = list(result.bundleDict.keys())[0]
    result.bundleDict[key]
    print(name, result.bundleDict[key].summaryValues)